> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [5]:
### Escreva sua resposta aqui
from google.colab import files
uploaded = files.upload()


Saving salaries.csv to salaries.csv


In [26]:
import pandas as pd

# Leitura do CSV
df = pd.read_csv("salaries.csv")
df.head()


,FIRST NAME,LAST NAME,SEX,DOJ,CURRENT DATE,DESIGNATION,AGE,SALARY,UNIT,LEAVES USED,LEAVES REMAINING,RATINGS,PAST EXP
0,TOMASA,ARMEN,F,5-18-2014,01-07-2016,Analyst,21.0,44570,Finance,24.0,6.0,2.0,0
1,ANNIE,NaN,F,NaN,01-07-2016,Associate,NaN,89207,Web,NaN,13.0,NaN,7
2,OLIVE,ANCY,F,7-28-2014,01-07-2016,Analyst,21.0,40955,Finance,23.0,7.0,3.0,0
3,CHERRY,AQUILAR,F,04-03-2013,01-07-2016,Analyst,22.0,45550,IT,22.0,8.0,3.0,0
4,LEON,ABOULAHOUD,M,11-20-2014,01-07-2016,Analyst,NaN,43161,Operations,27.0,3.0,NaN,3


#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [27]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine, Column, Integer, Float, String, Date
from sqlalchemy.orm import declarative_base
import enum

Base = declarative_base()

class SexEnum(enum.Enum):
    M = "M"
    F = "F"

class DesignationEnum(enum.Enum):
    ANALYST = "Analyst"
    SR_ANALYST = "Sr Analyst"
    MANAGER = "Manager"
    AVP = "AVP"
    VP = "VP"
    SENIOR_MANAGER = "Senior Manager"

class UnitEnum(enum.Enum):
    IT = "IT"
    FINANCE = "Finance"
    MARKETING = "Marketing"
    HR = "HR"
    OPERATIONS = "Operations"
    ADMIN = "Admin"

class Employee(Base):
    __tablename__ = 'employees'

    id = Column(Integer, primary_key=True, autoincrement=True)
    first_name = Column(String)
    last_name = Column(String)
    sex = Column(String)
    doj = Column(Date)
    current_date = Column(Date)
    designation = Column(String)
    age = Column(Integer)
    salary = Column(Float)
    unit = Column(String)
    leaves_used = Column(Integer)
    leaves_remaining = Column(Integer)
    ratings = Column(Float)
    past_exp = Column(Float)


#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [28]:
### Escreva sua resposta aqui
from sqlalchemy import create_engine

# Conexão com o banco SQLite
engine = create_engine('sqlite:///salarios.db')

print("Conexão com o banco 'salarios.db' criada com sucesso!")


Conexão com o banco 'salarios.db' criada com sucesso!


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [29]:
### Escreva sua resposta aqui
Base.metadata.create_all(engine)
print("Tabelas criadas com sucesso no banco 'salarios.db'!")

Tabelas criadas com sucesso no banco 'salarios.db'!


#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [30]:
### Escreva sua resposta aqui
import pandas as pd
df = pd.read_csv("salaries.csv")

In [31]:
df["DOJ"] = pd.to_datetime(df["DOJ"])
df["CURRENT DATE"] = pd.to_datetime(df["CURRENT DATE"])

In [32]:
df = df.rename(columns={
    "FIRST NAME": "first_name",
    "LAST NAME": "last_name",
    "SEX": "sex",
    "DOJ": "doj",
    "CURRENT DATE": "current_date",
    "DESIGNATION": "designation",
    "AGE": "age",
    "SALARY": "salary",
    "UNIT": "unit",
    "LEAVES USED": "leaves_used",
    "LEAVES REMAINING": "leaves_remaining",
    "RATINGS": "ratings",
    "PAST EXP": "past_exp"
})

In [33]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///salarios.db')

# Inserindo os dados
df.to_sql('employees', con=engine, if_exists='append', index=False)

print("Dados inseridos com sucesso no banco 'salarios.db'!")

Dados inseridos com sucesso no banco 'salarios.db'!


#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [34]:
### Execute aqui sua query SQL com SQLAlchemy
from sqlalchemy import text

query = text("""
    SELECT
        designation,
        MIN(salary / 12) AS salario_min_mensal,
        MAX(salary / 12) AS salario_max_mensal,
        AVG(salary / 12) AS salario_medio_mensal
    FROM employees
    GROUP BY designation
""")

with engine.connect() as conn:
    result = conn.execute(query)

    # Exibindo os resultados
    for row in result:
        print(row)


('Analyst', 3333.4166666666665, 4165.0, 3751.6759876859915)
('Associate', 5846.166666666667, 8300.25, 7266.915094339625)
('Director', 17832.25, 32342.666666666668, 23914.265625000004)
('Manager', 8343.666666666666, 12407.5, 10522.716049382716)
('Senior Analyst', 4170.333333333333, 5830.5, 4991.778792134831)
('Senior Manager', 12614.416666666666, 16631.416666666668, 14888.689516129034)


In [35]:
### Execute aqui sua query SQL com SQLAlchemy + Pandas
import pandas as pd

query = """
    SELECT
        designation,
        MIN(salary / 12) AS salario_min_mensal,
        MAX(salary / 12) AS salario_max_mensal,
        AVG(salary / 12) AS salario_medio_mensal
    FROM employees
    GROUP BY designation
"""

df_result = pd.read_sql_query(query, con=engine)
df_result

,designation,salario_min_mensal,salario_max_mensal,salario_medio_mensal
0,Analyst,3333.416667,4165.000000,3751.675988
1,Associate,5846.166667,8300.250000,7266.915094
2,Director,17832.250000,32342.666667,23914.265625
3,Manager,8343.666667,12407.500000,10522.716049
4,Senior Analyst,4170.333333,5830.500000,4991.778792
5,Senior Manager,12614.416667,16631.416667,14888.689516


In [37]:
### Execute aqui sua query com SQLAlchemy ORM
from sqlalchemy.orm import Session
from sqlalchemy import select, func

session = Session(engine)

stmt = (
    select(
        Employee.designation,
        func.min(Employee.salary / 12).label("salario_min_mensal"),
        func.max(Employee.salary / 12).label("salario_max_mensal"),
        func.avg(Employee.salary / 12).label("salario_medio_mensal"),
    )
    .group_by(Employee.designation)
)
result = session.execute(stmt).all()

for row in result:
    print({
        "designation": row.designation,
        "salario_min_mensal": row.salario_min_mensal,
        "salario_max_mensal": row.salario_max_mensal,
        "salario_medio_mensal": row.salario_medio_mensal,
    })


{'designation': 'Analyst', 'salario_min_mensal': 3333.4166666666665, 'salario_max_mensal': 4165.0, 'salario_medio_mensal': 3751.6759876859915}
{'designation': 'Associate', 'salario_min_mensal': 5846.166666666667, 'salario_max_mensal': 8300.25, 'salario_medio_mensal': 7266.915094339625}
{'designation': 'Director', 'salario_min_mensal': 17832.25, 'salario_max_mensal': 32342.666666666668, 'salario_medio_mensal': 23914.265625000004}
{'designation': 'Manager', 'salario_min_mensal': 8343.666666666666, 'salario_max_mensal': 12407.5, 'salario_medio_mensal': 10522.716049382716}
{'designation': 'Senior Analyst', 'salario_min_mensal': 4170.333333333333, 'salario_max_mensal': 5830.5, 'salario_medio_mensal': 4991.778792134831}
{'designation': 'Senior Manager', 'salario_min_mensal': 12614.416666666666, 'salario_max_mensal': 16631.416666666668, 'salario_medio_mensal': 14888.689516129034}
